In [24]:
import torchvision.models as models
import pydicom
from torch.utils.data import Dataset,DataLoader
import os
import pylab
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torch
import cv2
from openpyxl import Workbook

In [44]:
'''
0->硬膜外出血 epidural: 223
1->健康 healthy: 977
2->腦實質性出血 intraparenchymal: 807
3->腦室內出血 intraventricular: 712
4->蛛網膜下腔出血 subarachnoid: 717
5->硬膜下出血 subdural: 794
'''

class_dict={
    0:'epidural',
    1:'healthy',
    2:'intraparenchymal',
    3:'intraventricular',
    4:'subarachnoid',
    5:'subdural'
}

In [2]:
device = torch.cuda.current_device()
print('device:',device)

device: 0


In [46]:
class ICHDataset(Dataset):
    def __init__(self,root,train=True):
        self.train=train
        self.labels=[]
        self.filename_list=[]
        
        if train:
            all_dirs=os.listdir(root)

            label=0
            for dir in all_dirs:
                path=os.path.join(root,dir)
                all_files=os.listdir(path)
                for i,file in enumerate(all_files):
                    self.filename_list.append(os.path.join(path,file))
                    self.labels.append(label)
                label+=1
    
        if not train:
            all_files=os.listdir(root)
            for file in all_files:
                self.filename_list.append(os.path.join(root,file))
            
            
    def __getitem__(self,idx):
        filename=self.filename_list[idx]
        pixel_arr=pydicom.read_file(filename).pixel_array
        pixel_arr=cv2.resize(pixel_arr,(512,512))
        pixel_arr=np.float32(pixel_arr)
        pixel_arr=cv2.cvtColor(pixel_arr, cv2.COLOR_GRAY2BGR)
        transform = transforms.Compose([            
            transforms.ToTensor()
        ])
        data=transform(pixel_arr)
        if self.train:
            target=self.labels[idx]
            return data,target
        else:
            return data
        
    def __len__(self):
        return len(self.filename_list)
                


In [47]:
epochs=23
lr=0.01
batch_size=16
criterion=nn.CrossEntropyLoss()
num_classes=6

In [48]:
trainset=ICHDataset('TrainingData')
dataloader=DataLoader(trainset,batch_size=batch_size,shuffle=True)
resnet18 = models.resnet18()
print(trainset[0][0])

tensor([[[ 22.,  20.,  21.,  ...,  21.,  22.,  21.],
         [ 22.,  21.,  21.,  ...,  21.,  21.,  22.],
         [ 23.,  23.,  20.,  ...,  20.,  20.,  21.],
         ...,
         [255., 212., 179.,  ...,  53.,  52.,  51.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[ 22.,  20.,  21.,  ...,  21.,  22.,  21.],
         [ 22.,  21.,  21.,  ...,  21.,  21.,  22.],
         [ 23.,  23.,  20.,  ...,  20.,  20.,  21.],
         ...,
         [255., 212., 179.,  ...,  53.,  52.,  51.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]],

        [[ 22.,  20.,  21.,  ...,  21.,  22.,  21.],
         [ 22.,  21.,  21.,  ...,  21.,  21.,  22.],
         [ 23.,  23.,  20.,  ...,  20.,  20.,  21.],
         ...,
         [255., 212., 179.,  ...,  53.,  52.,  51.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.]]]

In [14]:
resnet18.fc=nn.Linear(512,num_classes)
resnet18=resnet18.to(device)
optimizer=optim.SGD(resnet18.parameters(),lr=lr,momentum=0.9,weight_decay=0.0005)

In [12]:
#Train
def train(model,dataloader):
    for epoch in range(epochs):
        correct=0.0
        running_loss=0.0 
        for times,data in enumerate(dataloader):
            optimizer.zero_grad()
            inputs,labels=data
            inputs,labels=inputs.to(device),labels.to(device)
            outputs=model(inputs)
            _, predict = torch.max(outputs.data, 1)
            correct+=(predict==labels).sum().item()

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            print('[%d/%d] [%d/%d] loss: %.3f' % (epoch+1, epochs, times, len(trainset)/batch_size, running_loss))
        
        accuracy=correct/len(trainset)*100
        print('Train accuracy: %.3f%%' % accuracy)

train(resnet18,dataloader)

[1/23] [0/375] loss: 1.621
[1/23] [1/375] loss: 3.641
[1/23] [2/375] loss: 5.321
[1/23] [3/375] loss: 7.205
[1/23] [4/375] loss: 8.853
[1/23] [5/375] loss: 10.495
[1/23] [6/375] loss: 12.065
[1/23] [7/375] loss: 13.591
[1/23] [8/375] loss: 15.220
[1/23] [9/375] loss: 16.881
[1/23] [10/375] loss: 18.646
[1/23] [11/375] loss: 20.614
[1/23] [12/375] loss: 22.702
[1/23] [13/375] loss: 24.579
[1/23] [14/375] loss: 26.187
[1/23] [15/375] loss: 28.349
[1/23] [16/375] loss: 30.432
[1/23] [17/375] loss: 32.159
[1/23] [18/375] loss: 33.871
[1/23] [19/375] loss: 35.503
[1/23] [20/375] loss: 37.258
[1/23] [21/375] loss: 39.199
[1/23] [22/375] loss: 40.951
[1/23] [23/375] loss: 42.679
[1/23] [24/375] loss: 44.576
[1/23] [25/375] loss: 46.197
[1/23] [26/375] loss: 47.777
[1/23] [27/375] loss: 49.574
[1/23] [28/375] loss: 51.460
[1/23] [29/375] loss: 53.170
[1/23] [30/375] loss: 54.872
[1/23] [31/375] loss: 56.591
[1/23] [32/375] loss: 58.490
[1/23] [33/375] loss: 60.084
[1/23] [34/375] loss: 61.793


[1/23] [271/375] loss: 467.657
[1/23] [272/375] loss: 469.206
[1/23] [273/375] loss: 470.988
[1/23] [274/375] loss: 472.776
[1/23] [275/375] loss: 474.357
[1/23] [276/375] loss: 475.875
[1/23] [277/375] loss: 477.581
[1/23] [278/375] loss: 479.198
[1/23] [279/375] loss: 480.928
[1/23] [280/375] loss: 482.986
[1/23] [281/375] loss: 484.831
[1/23] [282/375] loss: 486.507
[1/23] [283/375] loss: 488.017
[1/23] [284/375] loss: 489.681
[1/23] [285/375] loss: 491.475
[1/23] [286/375] loss: 493.186
[1/23] [287/375] loss: 494.654
[1/23] [288/375] loss: 496.307
[1/23] [289/375] loss: 498.031
[1/23] [290/375] loss: 499.682
[1/23] [291/375] loss: 501.312
[1/23] [292/375] loss: 502.907
[1/23] [293/375] loss: 504.607
[1/23] [294/375] loss: 506.269
[1/23] [295/375] loss: 508.334
[1/23] [296/375] loss: 509.983
[1/23] [297/375] loss: 511.565
[1/23] [298/375] loss: 513.131
[1/23] [299/375] loss: 514.789
[1/23] [300/375] loss: 516.426
[1/23] [301/375] loss: 518.199
[1/23] [302/375] loss: 519.775
[1/23] [

[2/23] [166/375] loss: 282.581
[2/23] [167/375] loss: 284.413
[2/23] [168/375] loss: 285.850
[2/23] [169/375] loss: 287.470
[2/23] [170/375] loss: 288.947
[2/23] [171/375] loss: 290.618
[2/23] [172/375] loss: 292.301
[2/23] [173/375] loss: 293.835
[2/23] [174/375] loss: 295.612
[2/23] [175/375] loss: 297.078
[2/23] [176/375] loss: 298.764
[2/23] [177/375] loss: 300.205
[2/23] [178/375] loss: 301.747
[2/23] [179/375] loss: 303.403
[2/23] [180/375] loss: 304.868
[2/23] [181/375] loss: 306.492
[2/23] [182/375] loss: 307.876
[2/23] [183/375] loss: 309.475
[2/23] [184/375] loss: 311.224
[2/23] [185/375] loss: 312.894
[2/23] [186/375] loss: 314.612
[2/23] [187/375] loss: 316.296
[2/23] [188/375] loss: 317.605
[2/23] [189/375] loss: 319.199
[2/23] [190/375] loss: 320.695
[2/23] [191/375] loss: 322.249
[2/23] [192/375] loss: 323.794
[2/23] [193/375] loss: 325.475
[2/23] [194/375] loss: 327.082
[2/23] [195/375] loss: 328.482
[2/23] [196/375] loss: 330.128
[2/23] [197/375] loss: 331.822
[2/23] [

[3/23] [59/375] loss: 98.998
[3/23] [60/375] loss: 100.692
[3/23] [61/375] loss: 102.075
[3/23] [62/375] loss: 103.670
[3/23] [63/375] loss: 105.139
[3/23] [64/375] loss: 107.000
[3/23] [65/375] loss: 108.648
[3/23] [66/375] loss: 110.150
[3/23] [67/375] loss: 111.759
[3/23] [68/375] loss: 113.277
[3/23] [69/375] loss: 115.062
[3/23] [70/375] loss: 116.759
[3/23] [71/375] loss: 118.065
[3/23] [72/375] loss: 119.647
[3/23] [73/375] loss: 121.406
[3/23] [74/375] loss: 122.830
[3/23] [75/375] loss: 124.328
[3/23] [76/375] loss: 126.360
[3/23] [77/375] loss: 127.929
[3/23] [78/375] loss: 129.376
[3/23] [79/375] loss: 130.803
[3/23] [80/375] loss: 132.470
[3/23] [81/375] loss: 134.249
[3/23] [82/375] loss: 135.959
[3/23] [83/375] loss: 137.893
[3/23] [84/375] loss: 139.457
[3/23] [85/375] loss: 141.141
[3/23] [86/375] loss: 142.640
[3/23] [87/375] loss: 144.369
[3/23] [88/375] loss: 145.963
[3/23] [89/375] loss: 147.433
[3/23] [90/375] loss: 149.077
[3/23] [91/375] loss: 150.685
[3/23] [92/

[3/23] [326/375] loss: 534.449
[3/23] [327/375] loss: 536.313
[3/23] [328/375] loss: 538.127
[3/23] [329/375] loss: 539.883
[3/23] [330/375] loss: 541.474
[3/23] [331/375] loss: 543.239
[3/23] [332/375] loss: 544.677
[3/23] [333/375] loss: 546.468
[3/23] [334/375] loss: 548.096
[3/23] [335/375] loss: 549.625
[3/23] [336/375] loss: 551.304
[3/23] [337/375] loss: 552.900
[3/23] [338/375] loss: 554.730
[3/23] [339/375] loss: 556.068
[3/23] [340/375] loss: 557.671
[3/23] [341/375] loss: 559.206
[3/23] [342/375] loss: 561.014
[3/23] [343/375] loss: 563.018
[3/23] [344/375] loss: 564.552
[3/23] [345/375] loss: 565.939
[3/23] [346/375] loss: 567.358
[3/23] [347/375] loss: 569.016
[3/23] [348/375] loss: 570.647
[3/23] [349/375] loss: 572.104
[3/23] [350/375] loss: 573.640
[3/23] [351/375] loss: 575.349
[3/23] [352/375] loss: 576.958
[3/23] [353/375] loss: 578.310
[3/23] [354/375] loss: 579.916
[3/23] [355/375] loss: 581.615
[3/23] [356/375] loss: 583.089
[3/23] [357/375] loss: 584.905
[3/23] [

...

[20/23] [277/375] loss: 333.421
[20/23] [278/375] loss: 334.881
[20/23] [279/375] loss: 336.267
[20/23] [280/375] loss: 337.870
[20/23] [281/375] loss: 338.989
[20/23] [282/375] loss: 340.197
[20/23] [283/375] loss: 341.299
[20/23] [284/375] loss: 342.390
[20/23] [285/375] loss: 343.864
[20/23] [286/375] loss: 345.180
[20/23] [287/375] loss: 346.104
[20/23] [288/375] loss: 347.358
[20/23] [289/375] loss: 348.782
[20/23] [290/375] loss: 350.559
[20/23] [291/375] loss: 351.495
[20/23] [292/375] loss: 352.814
[20/23] [293/375] loss: 354.065
[20/23] [294/375] loss: 355.338
[20/23] [295/375] loss: 356.640
[20/23] [296/375] loss: 357.628
[20/23] [297/375] loss: 358.540
[20/23] [298/375] loss: 359.678
[20/23] [299/375] loss: 361.155
[20/23] [300/375] loss: 362.241
[20/23] [301/375] loss: 363.025
[20/23] [302/375] loss: 364.167
[20/23] [303/375] loss: 364.983
[20/23] [304/375] loss: 366.228
[20/23] [305/375] loss: 367.165
[20/23] [306/375] loss: 368.446
[20/23] [307/375] loss: 369.792
[20/23] 

[21/23] [165/375] loss: 180.593
[21/23] [166/375] loss: 181.498
[21/23] [167/375] loss: 182.707
[21/23] [168/375] loss: 183.875
[21/23] [169/375] loss: 185.157
[21/23] [170/375] loss: 186.434
[21/23] [171/375] loss: 187.931
[21/23] [172/375] loss: 188.943
[21/23] [173/375] loss: 189.748
[21/23] [174/375] loss: 190.826
[21/23] [175/375] loss: 191.748
[21/23] [176/375] loss: 193.025
[21/23] [177/375] loss: 194.125
[21/23] [178/375] loss: 195.252
[21/23] [179/375] loss: 196.349
[21/23] [180/375] loss: 197.455
[21/23] [181/375] loss: 198.637
[21/23] [182/375] loss: 199.435
[21/23] [183/375] loss: 200.581
[21/23] [184/375] loss: 201.458
[21/23] [185/375] loss: 202.183
[21/23] [186/375] loss: 203.182
[21/23] [187/375] loss: 203.881
[21/23] [188/375] loss: 205.179
[21/23] [189/375] loss: 206.701
[21/23] [190/375] loss: 207.785
[21/23] [191/375] loss: 208.928
[21/23] [192/375] loss: 210.316
[21/23] [193/375] loss: 211.285
[21/23] [194/375] loss: 212.303
[21/23] [195/375] loss: 213.312
[21/23] 

[22/23] [50/375] loss: 50.603
[22/23] [51/375] loss: 51.677
[22/23] [52/375] loss: 53.004
[22/23] [53/375] loss: 54.014
[22/23] [54/375] loss: 55.293
[22/23] [55/375] loss: 56.259
[22/23] [56/375] loss: 57.292
[22/23] [57/375] loss: 58.179
[22/23] [58/375] loss: 59.119
[22/23] [59/375] loss: 60.091
[22/23] [60/375] loss: 60.555
[22/23] [61/375] loss: 61.847
[22/23] [62/375] loss: 62.598
[22/23] [63/375] loss: 63.784
[22/23] [64/375] loss: 64.828
[22/23] [65/375] loss: 65.336
[22/23] [66/375] loss: 66.450
[22/23] [67/375] loss: 67.619
[22/23] [68/375] loss: 69.021
[22/23] [69/375] loss: 70.022
[22/23] [70/375] loss: 71.032
[22/23] [71/375] loss: 72.311
[22/23] [72/375] loss: 74.102
[22/23] [73/375] loss: 75.347
[22/23] [74/375] loss: 76.393
[22/23] [75/375] loss: 77.132
[22/23] [76/375] loss: 78.170
[22/23] [77/375] loss: 79.418
[22/23] [78/375] loss: 80.365
[22/23] [79/375] loss: 81.348
[22/23] [80/375] loss: 82.437
[22/23] [81/375] loss: 83.338
[22/23] [82/375] loss: 84.501
[22/23] [8

[22/23] [310/375] loss: 322.793
[22/23] [311/375] loss: 323.617
[22/23] [312/375] loss: 325.097
[22/23] [313/375] loss: 326.242
[22/23] [314/375] loss: 327.272
[22/23] [315/375] loss: 328.350
[22/23] [316/375] loss: 329.135
[22/23] [317/375] loss: 330.004
[22/23] [318/375] loss: 330.712
[22/23] [319/375] loss: 331.842
[22/23] [320/375] loss: 332.756
[22/23] [321/375] loss: 334.120
[22/23] [322/375] loss: 335.223
[22/23] [323/375] loss: 336.351
[22/23] [324/375] loss: 337.578
[22/23] [325/375] loss: 338.384
[22/23] [326/375] loss: 339.565
[22/23] [327/375] loss: 340.740
[22/23] [328/375] loss: 341.864
[22/23] [329/375] loss: 342.817
[22/23] [330/375] loss: 343.998
[22/23] [331/375] loss: 345.059
[22/23] [332/375] loss: 346.355
[22/23] [333/375] loss: 347.643
[22/23] [334/375] loss: 349.101
[22/23] [335/375] loss: 350.097
[22/23] [336/375] loss: 351.139
[22/23] [337/375] loss: 352.581
[22/23] [338/375] loss: 353.871
[22/23] [339/375] loss: 355.071
[22/23] [340/375] loss: 356.303
[22/23] 

[23/23] [198/375] loss: 186.456
[23/23] [199/375] loss: 187.587
[23/23] [200/375] loss: 188.622
[23/23] [201/375] loss: 189.470
[23/23] [202/375] loss: 190.028
[23/23] [203/375] loss: 190.802
[23/23] [204/375] loss: 191.728
[23/23] [205/375] loss: 192.963
[23/23] [206/375] loss: 193.918
[23/23] [207/375] loss: 194.584
[23/23] [208/375] loss: 195.189
[23/23] [209/375] loss: 196.297
[23/23] [210/375] loss: 197.275
[23/23] [211/375] loss: 198.396
[23/23] [212/375] loss: 199.532
[23/23] [213/375] loss: 200.324
[23/23] [214/375] loss: 201.239
[23/23] [215/375] loss: 202.321
[23/23] [216/375] loss: 203.199
[23/23] [217/375] loss: 204.178
[23/23] [218/375] loss: 205.209
[23/23] [219/375] loss: 206.107
[23/23] [220/375] loss: 207.564
[23/23] [221/375] loss: 209.030
[23/23] [222/375] loss: 209.552
[23/23] [223/375] loss: 210.436
[23/23] [224/375] loss: 211.616
[23/23] [225/375] loss: 212.680
[23/23] [226/375] loss: 213.671
[23/23] [227/375] loss: 214.479
[23/23] [228/375] loss: 215.316
[23/23] 

In [21]:
def test(model,dataloader):
    res=[]
    for data in dataloader:
        data=data.to(device)
        outputs=model(data)
        _, predict = torch.max(outputs.data, 1)
        predict=predict.tolist()
        res+=predict
    return res

testset=ICHDataset('TestingData',train=False)
dataloader=DataLoader(testset,batch_size,shuffle=False)
res=test(resnet18,dataloader)
print(res)

[5, 5, 5, 5, 0, 5, 5, 5, 0, 5, 0, 0, 0, 5, 0, 5, 5, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5, 0, 5, 5, 0, 5, 5, 0, 5, 5, 0, 1, 0, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 0, 5, 0, 5, 5, 0, 0, 5, 5, 0, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 0, 5, 5, 5, 5, 5, 5, 0, 5, 0, 1, 5, 5, 5, 1, 5, 0, 5, 0, 1, 5, 5, 5, 5, 5, 0, 5, 0, 5, 1, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 0, 0, 0, 0, 5, 5, 5, 5, 1, 5, 5, 5, 1, 5, 1, 0, 5, 5, 5, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 0, 5, 5, 5, 0, 1, 5, 0, 5, 5, 0, 5, 0, 1, 5, 5, 5, 0, 5, 5, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 1, 5, 5, 0, 5, 5, 5, 0, 5, 5, 5, 0, 5, 5, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 0, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 0, 5, 0, 5, 5, 5, 0, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 5, 5, 0, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 0, 5, 5, 5, 1, 5, 5, 5, 0, 5, 5, 5, 5, 5, 0, 5, 5, 1, 5, 0, 5, 0, 5, 5, 5, 5, 0, 5, 1, 5, 5, 

In [45]:
# write excel file
wb=Workbook()
ws=wb.active
for i in range(len(testset)):
    val=testset.filename_list[i].split('/')[1]
    val=val.split('.')[0]
    ws.cell(i+1,1).value=val
    ws.cell(i+1,2).value=class_dict[res[i]]
wb.save('testing_result.xlsx')